# Nearest Neighbor product based Collaborative Filtering

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [2]:
combined=pd.read_csv("Combined.csv")

In [ ]:
combined.head()

,Unnamed: 0,order_id,customer_unique_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,seller_lat,seller_lng,customer_city,customer_state,customer_lat,customer_lng,order_delivered_customer_time,dist,product_category_name_english,review_score
0,0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,1.0,4244733e06e7ecb4970a6e2683c13e61,...,-22.498183,-44.123614,campos dos goytacazes,RJ,-21.758076,-41.312633,104972,301.005664,기타,5.0
1,1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,1.0,e5f2d52b802189ee658865ca93d83a8f,...,-23.566258,-46.518417,santa fe do sul,SP,-20.212393,-50.941471,696564,589.274140,유아/애완용품,4.0
2,2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,1.0,c777355d18b72b67abbeef9df44fd0fd,...,-22.264094,-46.158564,para de minas,MG,-19.860439,-44.597972,520948,312.495046,가구/인테리어,5.0
3,3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,1.0,7634da152a4610f1595efa32f14722fc,...,-20.548228,-47.395897,atibaia,SP,-23.144923,-46.539830,345879,301.951753,뷰티/건강,4.0
4,4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,1.0,ac6c3623068f30de03045865e4e10089,...,-22.931427,-53.133759,varzea paulista,SP,-23.249008,-46.824961,1148182,646.221788,가구/인테리어,5.0


In [4]:
df1=pd.read_csv("combined.csv",usecols=['product_id','customer_unique_id','product_category_name_english','review_score'])

In [5]:
df1.head()

,customer_unique_id,product_id,product_category_name_english,review_score
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,기타,5.0
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,유아/애완용품,4.0
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,가구/인테리어,5.0
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,뷰티/건강,4.0
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,가구/인테리어,5.0


Combine total product review and count the total reviews

In [ ]:
combine_product_review = df1.dropna(axis = 0, subset = ['product_category_name_english'])
product_reviewCount = (combine_product_review.
     groupby(by = ['product_category_name_english'])['review_score'].
     count().
     reset_index().
     rename(columns = {'review_score': 'totalReviewCount'})
     [['product_category_name_english','totalReviewCount']]
    )
product_reviewCount.head()

,product_category_name_english,totalReviewCount
0,가구/인테리어,14885
1,기타,8628
2,도서/교육,852
3,미디어/엔터테인먼트,1260
4,뷰티/건강,12795


# After merging Total review count column into dataset

In [ ]:
review_with_totalReviewCount  = combine_product_review.merge(product_reviewCount, left_on = 'product_category_name_english', right_on = 'product_category_name_english', how = 'left')
review_with_totalReviewCount.head()

,customer_unique_id,product_id,product_category_name_english,review_score,totalReviewCount
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,기타,5.0,8628
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,유아/애완용품,4.0,8825
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,가구/인테리어,5.0,14885
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,뷰티/건강,4.0,12795
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,가구/인테리어,5.0,14885


In [8]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(product_reviewCount['totalReviewCount'].describe())

count      10.000
mean     8350.600
std      5796.878
min       852.000
25%      3056.250
50%      8726.500
75%     11950.000
max     17281.000
Name: totalReviewCount, dtype: float64


filtering out products that have less than 100 reviews 

In [ ]:
popularity_threshold =100
review_popular_product= review_with_totalReviewCount.query('totalReviewCount >= @popularity_threshold')
review_popular_product.head()

,customer_unique_id,product_id,product_category_name_english,review_score,totalReviewCount
0,3ce436f183e68e07877b285a838db11a,4244733e06e7ecb4970a6e2683c13e61,기타,5.000,8628
1,f6dd3ec061db4e3987629fe6b26e5cce,e5f2d52b802189ee658865ca93d83a8f,유아/애완용품,4.000,8825
2,6489ae5e4333f3693df5ad4372dab6d3,c777355d18b72b67abbeef9df44fd0fd,가구/인테리어,5.000,14885
3,d4eb9395c8c0431ee92fce09860c5a06,7634da152a4610f1595efa32f14722fc,뷰티/건강,4.000,12795
4,58dbd0b2d70206bf40e62cd34e84d795,ac6c3623068f30de03045865e4e10089,가구/인테리어,5.000,14885


In [10]:
review_popular_product.shape

(84096, 5)

# create a Pivot matrix

In [ ]:
product_features_df=review_popular_product.pivot_table(index='product_category_name_english',columns='customer_unique_id',values='review_score').fillna(0)
product_features_df.head()

customer_unique_id,00012a2ce6f8dcda20d059ce98491703,000161a058600d5901f007fab4c27140,0001fd6190edaaf884bcaf3d49edf079,0002414f95344307404f0ace7a26f1d5,0004164d20a9e969af783496f3408652,000419c5494106c306a97b5635748086,00046a560d407e99b969756e0b10f282,00050bf6e01e69d5c0fd612f1bcfb69c,000598caf2ef4117407665ac33275130,00062b33cb9f6fe976afdcff967ea74d,...,fff5169e583fd07fac9fec88962f189d,fff55ba8dffa552b6fdfd86c2e806459,fff5dd22d522cf28a902185817642a2e,fff675a0d5924b9162b4a1bf410466cd,fff7466a253c0e59499ea943462c10f9,fff93c1da78dafaaa304ff032abc6205,fffa0238b217e18a8adeeda0669923a3,fffcb937e9dd47a13f05ecb8290f4d3e,fffeda5b6d849fbd39689bb92087f431,ffffa3172527f765de70084a7e53aae8
product_category_name_english,,,,,,,,,,,,,,,,,,,,,
가구/인테리어,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000
기타,0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
도서/교육,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,5.000,0.000,5.000,0.000,0.000
미디어/엔터테인먼트,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
뷰티/건강,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,1.000,3.000,0.000,0.000,5.000,0.000,5.000,0.000,0.000,0.000


In [12]:
from scipy.sparse import csr_matrix

product_features_df_matrix = csr_matrix(product_features_df.values)

In [13]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(product_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [14]:
product_features_df.shape

(10, 73315)

# choose a random product

In [ ]:
query_index = np.random.choice(product_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(product_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

8


# Based on that product what we have choosen 5 nearest distance product get recommended

In [16]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(product_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, product_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 전자제품/기술:

1: 기타, with distance of 0.9989063982445366:
2: 가구/인테리어, with distance of 0.9990783569046878:
3: 패션/액세서리, with distance of 0.9990886430870719:
4: 식품/음료, with distance of 0.9995354699531809:
5: 뷰티/건강, with distance of 0.9995433563452749:
